In [35]:
import pandas as pd
import numpy as np

from sklearn import model_selection, preprocessing, linear_model, naive_bayes
from sklearn import model_selection, preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn import decomposition, ensemble
import pandas, xgboost

#from keras.preprocessing import text, sequence
#from keras import layers, models, optimizers
from sklearn.metrics import roc_auc_score

In [3]:
#loading the data

train_text = pd.read_json("test_instances.json", lines=True)
train_truth = pd.read_json("test_truth.json", lines=True)
test_text = pd.read_json("train_instances.json", lines=True)
test_truth = pd.read_json("train_truth.json", lines=True)

### Loading data and combining

In [4]:
train = pd.merge(train_text,train_truth, on = 'id')
train.head()

,id,postMedia,postText,postTimestamp,targetCaptions,targetDescription,targetKeywords,targetParagraphs,targetTitle,truthClass,truthJudgments,truthMean,truthMedian,truthMode
0,608310377143799808,[],[Apple's iOS 9 'App thinning' feature will giv...,Tue Jun 09 16:31:10 +0000 2015,['App thinning' will be supported on Apple's i...,'App thinning' will be supported on Apple's iO...,"Apple,gives,gigabytes,iOS,9,app,thinning,featu...",[Paying for a 64GB phone only to discover that...,Apple gives back gigabytes: iOS 9 'app thinnin...,no-clickbait,"[0.0, 0.6666667, 0.0, 0.33333334000000003, 0.0]",0.200000,0.000000,0.000000
1,609297109095972864,[media/609297109095972864.jpg],[RT @kenbrown12: Emerging market investors are...,Fri Jun 12 09:52:05 +0000 2015,"[Stocks Fall as Investors Watch Central Banks,...",Global investors have yanked $9.3 billion from...,"emerging market,emerging markets,em flows,em i...","[Emerging markets are out of favor., Global in...",Emerging Markets Suffer Largest Outflow in Sev...,no-clickbait,"[0.6666667, 0.0, 0.0, 0.0, 0.0]",0.133333,0.000000,0.000000
2,609504474621612032,[],[U.S. Soccer should start answering tough ques...,Fri Jun 12 23:36:05 +0000 2015,[US to vote for Ali in FIFA election and not B...,A U.S. Senator's scathing letter questioned U....,,"[WINNIPEG, Manitoba – The bubble U.S. Soccer i...",U.S. Soccer should start answering tough quest...,clickbait,"[0.33333334000000003, 0.6666667, 1.0, 0.0, 0.6...",0.533333,0.666667,0.666667
3,609748367049105408,[],[How theme parks like Disney World left the mi...,Sat Jun 13 15:45:13 +0000 2015,"[Some 1,000 persons turned out in Albuquerque,...","America's top family vacation spots, like the ...","disney, disney world, disney ticket prices, di...",[When Walt Disney World opened in an Orlando s...,How theme parks like Disney World left the mid...,no-clickbait,"[1.0, 0.0, 0.33333334000000003, 0.333333340000...",0.466667,0.333333,0.333333
4,608688782821453824,[media/608688782821453825.jpg],[Could light bulbs hurt your health? One compa...,Wed Jun 10 17:34:49 +0000 2015,[Electric lights have made the world safer and...,One company will put a health notice on all th...,"health, Should there be warning labels on your...",[(CNN)The light bulb always makes the world's ...,Warning labels on your light bulbs,clickbait,"[1.0, 0.33333334000000003, 0.6666667, 0.333333...",0.666667,0.666667,1.000000


In [5]:
test = pd.merge(test_text,test_truth, on = 'id')
test.head()

,id,postMedia,postText,postTimestamp,targetCaptions,targetDescription,targetKeywords,targetParagraphs,targetTitle,truthClass,truthJudgments,truthMean,truthMedian,truthMode
0,858462320779026432,[],[UK’s response to modern slavery leaving victi...,Sat Apr 29 23:25:41 +0000 2017,[modern-slavery-rex.jpg],“Inexcusable” failures in the UK’s system for ...,"modern slavery, Department For Work And Pensio...",[Thousands of modern slavery victims have not ...,‘Inexcusable’ failures in UK’s response to mod...,no-clickbait,"[0.33333333330000003, 0.0, 0.33333333330000003...",0.133333,0.000000,0.000000
1,858421020331560960,[],[this is good],Sat Apr 29 20:41:34 +0000 2017,"[In this July 1, 2010 file photo, Dr. Charmain...",President Donald Trump has appointed pro-life ...,"Americans United for Life, Dr. Charmaine Yoest...",[President Donald Trump has appointed the pro-...,Donald Trump Appoints Pro-Life Advocate as Ass...,clickbait,"[1.0, 1.0, 1.0, 1.0, 1.0]",1.000000,1.000000,1.000000
2,858368123753435136,[],"[The ""forgotten"" Trump roast: Relive his bruta...",Sat Apr 29 17:11:23 +0000 2017,[President Trump will not attend this year's W...,President Trump won't be at this year's White ...,"trump whcd, whcd, white house correspondents d...",[When the White House correspondents’ dinner i...,The ‘forgotten’ Trump roast: Relive his brutal...,no-clickbait,"[0.33333333330000003, 1.0, 0.33333333330000003...",0.466667,0.333333,0.333333
3,858323428260139008,[],[Meet the happiest #dog in the world!],Sat Apr 29 14:13:46 +0000 2017,"[Maru , Maru, Maru, Maru, Maru]","The article is about Maru, a husky dog who has...","Maru, husky, dogs, pandas, furball, instagram",[Adorable is probably an understatement. This ...,"Meet The Happiest Dog In The World, Maru The H...",clickbait,"[1.0, 0.6666666666000001, 1.0, 1.0, 1.0]",0.933333,1.000000,1.000000
4,858283602626347008,[],[Tokyo's subway is shut down amid fears over a...,Sat Apr 29 11:35:31 +0000 2017,[All nine lines of Tokyo's subway system were ...,"The temporary suspension, which lasted ten min...","Tokyo,subway,shut,fears,North,Korean,attack",[One of Tokyo's major subways systems says it ...,Tokyo's subway is shut down amid fears over an...,no-clickbait,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.000000,0.000000,0.000000


In [6]:
train = train[['postText','truthClass']]
test = test[['postText','truthClass']]
train['postText']=train['postText'].astype(str).str.replace('\[|\]|\'|\"', '')
test['postText']=test['postText'].astype(str).str.replace('\[|\]|\'|\"', '')

In [7]:
train.head()

,postText,truthClass
0,Apples iOS 9 App thinning feature will give yo...,no-clickbait
1,RT @kenbrown12: Emerging market investors are ...,no-clickbait
2,U.S. Soccer should start answering tough quest...,clickbait
3,How theme parks like Disney World left the mid...,no-clickbait
4,Could light bulbs hurt your health? One compan...,clickbait


In [8]:
# removing stop words is not working since 
# from nltk.corpus import stopwords
# stop = stopwords.words('english')

In [9]:
# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train['truthClass'] = encoder.fit_transform(train['truthClass'])
test['truthClass'] = encoder.fit_transform(test['truthClass'])

In [10]:
test.head()

,postText,truthClass
0,UK’s response to modern slavery leaving victim...,1
1,this is good,0
2,The forgotten Trump roast: Relive his brutal 2...,1
3,Meet the happiest #dog in the world!,0
4,Tokyos subway is shut down amid fears over an ...,1


In [11]:
train_x = train['postText']
train_y = train['truthClass']
test_x = test['postText']
test_y = test['truthClass']

In [12]:
print(train_x.shape,train_y.shape, test_x.shape,test_y .shape)

(2459,) (2459,) (19538,) (19538,)


### Feature engineering and preprocessing

In [13]:
# create a count vectorizer object 

count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(train_x)

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xtest_count =  count_vect.transform(test_x)

In [14]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(train_x)

xtrain_tfidf =  tfidf_vect.transform(train_x)
xtest_tfidf =  tfidf_vect.transform(test_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(train_x)

xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xtest_tfidf_ngram =  tfidf_vect_ngram.transform(test_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(train_x)

xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xtest_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(test_x)

In [23]:
# train a LDA Model
lda_model = decomposition.LatentDirichletAllocation(n_components=10, learning_method='online', max_iter=20)
X_topics = lda_model.fit_transform(xtrain_count)
topic_word = lda_model.components_ 
vocab = count_vect.get_feature_names()

# view the topic models
n_top_words = 10
topic_summaries = []
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    topic_summaries.append(' '.join(topic_words))

In [24]:
topic_summaries

['clinton hillary rally high on college video service open instagram',
 'the a of to in rt and on as for',
 'with and of lebron james a video after in his',
 'the rt to in of s is for at with',
 'to first is what police know via it like rt',
 'the to and of is a your up for this',
 'new gets york a state in prison reports every for',
 'to rt in on for of year people after the',
 'live his a report watch long questions he after hope',
 'with to say be 2016 rt mom her run dont']

In [25]:
print(train_x.shape,train_y.shape)

(2459,) (2459,)


#### Model testing

In [29]:
def train_model(classifier, feature_vector_train, label, feature_vector_test, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_test)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return roc_auc_score(predictions, test_y)

In [30]:
# Naive Bayes on Count Vectors
auc = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xtest_count)
print ("NB, Count Vectors: ", auc)

# Naive Bayes on Word Level TF IDF Vectors
auc = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xtest_tfidf)
print ("NB, WordLevel TF-IDF: ", auc)

# Naive Bayes on Ngram Level TF IDF Vectors
auc = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xtest_tfidf_ngram)
print ("NB, N-Gram Vectors: ", auc)

# Naive Bayes on Character Level TF IDF Vectors
auc = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xtest_tfidf_ngram_chars)
print ("NB, CharLevel Vectors: ", auc)

NB, Count Vectors:  0.6572797349281626
NB, WordLevel TF-IDF:  0.7362457938531912
NB, N-Gram Vectors:  0.6784650327003756
NB, CharLevel Vectors:  0.7563576097353598


In [31]:
# Linear Classifier on Count Vectors
auc = train_model(linear_model.LogisticRegression(), xtrain_count, train_y, xtest_count)
print ("LR, Count Vectors: ", auc)

# Linear Classifier on Word Level TF IDF Vectors
auc = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xtest_tfidf)
print ("LR, WordLevel TF-IDF: ", auc)

# Linear Classifier on Ngram Level TF IDF Vectors
auc = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xtest_tfidf_ngram)
print ("LR, N-Gram Vectors: ", auc)

# Linear Classifier on Character Level TF IDF Vectors
auc = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xtest_tfidf_ngram_chars)
print ("LR, CharLevel Vectors: ", auc)

LR, Count Vectors:  0.6679062532627703
LR, WordLevel TF-IDF:  0.7171636995117197
LR, N-Gram Vectors:  0.689351151355335
LR, CharLevel Vectors:  0.7254813496447201


In [32]:
# RF on Count Vectors
auc = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xtest_count)
print ("RF, Count Vectors: ", auc)

# RF on Word Level TF IDF Vectors
auc = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, xtest_tfidf)
print ("RF, WordLevel TF-IDF: ", auc)

RF, Count Vectors:  0.666227494156121
RF, WordLevel TF-IDF:  0.623548589572538


In [36]:
# Extereme Gradient Boosting on Count Vectors
auc = train_model(xgboost.XGBClassifier(), xtrain_count.tocsc(), train_y, xtest_count.tocsc())
print ("Xgb, Count Vectors: ", auc)

# Extereme Gradient Boosting on Word Level TF IDF Vectors
auc = train_model(xgboost.XGBClassifier(), xtrain_tfidf.tocsc(), train_y, xtest_tfidf.tocsc())
print ("Xgb, WordLevel TF-IDF: ", auc)

# Extereme Gradient Boosting on Character Level TF IDF Vectors
auc = train_model(xgboost.XGBClassifier(), xtrain_tfidf_ngram_chars.tocsc(), train_y, xtest_tfidf_ngram_chars.tocsc())
print ("Xgb, CharLevel Vectors: ", auc)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Xgb, Count Vectors:  0.6941378785557214


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Xgb, WordLevel TF-IDF:  0.6858915548462214
Xgb, CharLevel Vectors:  0.6962049670767736


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [37]:
print(train_x.shape,train_y.shape)

(2459,) (2459,)


#### As showen above, there is big change of auc score improving after tuning, now we tune XGBoost.

In [38]:
xgb1 = xgboost.XGBClassifier(
learning_rate =0.1,
n_estimators=1000,
max_depth=5,
min_child_weight=1,
gamma=0,
subsample=0.8,
colsample_bytree=0.8,
objective= 'binary:logistic',
nthread=4,
scale_pos_weight=1,
seed=27)

In [39]:
auc = train_model(xgb1, xtrain_tfidf_ngram_chars.tocsc(), train_y, xtest_tfidf_ngram_chars.tocsc())
print ("Xgb, CharLevel Vectors: ", auc)

Xgb, CharLevel Vectors:  0.7006844056001513


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [40]:
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)}

gsearch1 = GridSearchCV(estimator = xgboost.XGBClassifier(learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch1.fit(xtrain_tfidf_ngram_chars.tocsc(), train_y)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.72045, std: 0.02307, params: {'max_depth': 3, 'min_child_weight': 1},
  mean: 0.72217, std: 0.02106, params: {'max_depth': 3, 'min_child_weight': 3},
  mean: 0.73090, std: 0.01903, params: {'max_depth': 3, 'min_child_weight': 5},
  mean: 0.72657, std: 0.02703, params: {'max_depth': 5, 'min_child_weight': 1},
  mean: 0.73072, std: 0.01921, params: {'max_depth': 5, 'min_child_weight': 3},
  mean: 0.72746, std: 0.02586, params: {'max_depth': 5, 'min_child_weight': 5},
  mean: 0.72586, std: 0.02442, params: {'max_depth': 7, 'min_child_weight': 1},
  mean: 0.72067, std: 0.02129, params: {'max_depth': 7, 'min_child_weight': 3},
  mean: 0.71760, std: 0.02215, params: {'max_depth': 7, 'min_child_weight': 5},
  mean: 0.72537, std: 0.02041, params: {'max_depth': 9, 'min_child_weight': 1},
  mean: 0.71855, std: 0.02133, params: {'max_depth': 9, 'min_child_weight': 3},
  mean: 0.72397, std: 0.01537, params: {'max_depth': 9, 'min_child_weight': 5}],
 {'max_depth': 3, 'min_child_weight': 5

In [42]:
param_test2 = {
 'max_depth':[2,3,4],
 'min_child_weight':[5,6,7]
}
gsearch2 = GridSearchCV(estimator = xgboost.XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=5,
 min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch2.fit(xtrain_tfidf_ngram_chars.tocsc(), train_y)
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.72032, std: 0.02653, params: {'max_depth': 2, 'min_child_weight': 5},
  mean: 0.71658, std: 0.02158, params: {'max_depth': 2, 'min_child_weight': 6},
  mean: 0.71646, std: 0.02615, params: {'max_depth': 2, 'min_child_weight': 7},
  mean: 0.73090, std: 0.01903, params: {'max_depth': 3, 'min_child_weight': 5},
  mean: 0.72552, std: 0.02386, params: {'max_depth': 3, 'min_child_weight': 6},
  mean: 0.72557, std: 0.02761, params: {'max_depth': 3, 'min_child_weight': 7},
  mean: 0.72324, std: 0.01821, params: {'max_depth': 4, 'min_child_weight': 5},
  mean: 0.71975, std: 0.02090, params: {'max_depth': 4, 'min_child_weight': 6},
  mean: 0.71691, std: 0.02366, params: {'max_depth': 4, 'min_child_weight': 7}],
 {'max_depth': 3, 'min_child_weight': 5},
 0.7308978140147809)

In [50]:
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = xgboost.XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=3,
 min_child_weight=5, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test3, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch3.fit(xtrain_tfidf_ngram_chars.tocsc(), train_y)
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.73090, std: 0.01903, params: {'gamma': 0.0},
  mean: 0.72500, std: 0.01631, params: {'gamma': 0.1},
  mean: 0.72545, std: 0.01703, params: {'gamma': 0.2},
  mean: 0.72833, std: 0.02619, params: {'gamma': 0.3},
  mean: 0.73130, std: 0.02441, params: {'gamma': 0.4}],
 {'gamma': 0.4},
 0.7313047863966906)

In [53]:
xgb2 = xgboost.XGBClassifier(
learning_rate =0.1,
n_estimators=1000,
max_depth=3,
min_child_weight=5,
gamma=0.4,
subsample=0.8,
colsample_bytree=0.8,
objective= 'binary:logistic',
nthread=4,
scale_pos_weight=1,
seed=27)

In [57]:
param_test4 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch4 = GridSearchCV(estimator = xgboost.XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=3,
 min_child_weight=5, gamma=0.4, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test4, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch4.fit(xtrain_tfidf_ngram_chars.tocsc(), train_y)
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.71618, std: 0.02087, params: {'colsample_bytree': 0.6, 'subsample': 0.6},
  mean: 0.72195, std: 0.02336, params: {'colsample_bytree': 0.6, 'subsample': 0.7},
  mean: 0.72846, std: 0.02557, params: {'colsample_bytree': 0.6, 'subsample': 0.8},
  mean: 0.71660, std: 0.02357, params: {'colsample_bytree': 0.6, 'subsample': 0.9},
  mean: 0.71443, std: 0.02719, params: {'colsample_bytree': 0.7, 'subsample': 0.6},
  mean: 0.72521, std: 0.02250, params: {'colsample_bytree': 0.7, 'subsample': 0.7},
  mean: 0.72965, std: 0.02475, params: {'colsample_bytree': 0.7, 'subsample': 0.8},
  mean: 0.71796, std: 0.02060, params: {'colsample_bytree': 0.7, 'subsample': 0.9},
  mean: 0.71713, std: 0.02208, params: {'colsample_bytree': 0.8, 'subsample': 0.6},
  mean: 0.72328, std: 0.02797, params: {'colsample_bytree': 0.8, 'subsample': 0.7},
  mean: 0.73203, std: 0.02490, params: {'colsample_bytree': 0.8, 'subsample': 0.8},
  mean: 0.71986, std: 0.02322, params: {'colsample_bytree': 0.8, 'subsample'

In [59]:
param_test5 = {
 'subsample':[i/100.0 for i in range(75,90,5)],
 'colsample_bytree':[i/100.0 for i in range(75,90,5)]
}

gsearch5 = GridSearchCV(estimator = xgboost.XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=3,
 min_child_weight=5, gamma=0.4, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test5, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch5.fit(xtrain_tfidf_ngram_chars.tocsc(), train_y)
gsearch5.grid_scores_, gsearch5.best_params_, gsearch5.best_score_

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.72914, std: 0.01965, params: {'colsample_bytree': 0.75, 'subsample': 0.75},
  mean: 0.72704, std: 0.02392, params: {'colsample_bytree': 0.75, 'subsample': 0.8},
  mean: 0.72581, std: 0.02557, params: {'colsample_bytree': 0.75, 'subsample': 0.85},
  mean: 0.71997, std: 0.02360, params: {'colsample_bytree': 0.8, 'subsample': 0.75},
  mean: 0.73203, std: 0.02490, params: {'colsample_bytree': 0.8, 'subsample': 0.8},
  mean: 0.72594, std: 0.02483, params: {'colsample_bytree': 0.8, 'subsample': 0.85},
  mean: 0.72404, std: 0.02609, params: {'colsample_bytree': 0.85, 'subsample': 0.75},
  mean: 0.72810, std: 0.01570, params: {'colsample_bytree': 0.85, 'subsample': 0.8},
  mean: 0.71614, std: 0.02276, params: {'colsample_bytree': 0.85, 'subsample': 0.85}],
 {'colsample_bytree': 0.8, 'subsample': 0.8},
 0.7320309429195571)

In [61]:
param_test6 = {
 'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}
gsearch6 = GridSearchCV(estimator = xgboost.XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=3,
 min_child_weight=5, gamma=0.4, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test6, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch6.fit(xtrain_tfidf_ngram_chars.tocsc(), train_y)
gsearch6.grid_scores_, gsearch6.best_params_, gsearch6.best_score_

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.73203, std: 0.02490, params: {'reg_alpha': 1e-05},
  mean: 0.73398, std: 0.02347, params: {'reg_alpha': 0.01},
  mean: 0.72521, std: 0.01679, params: {'reg_alpha': 0.1},
  mean: 0.72390, std: 0.02249, params: {'reg_alpha': 1},
  mean: 0.64926, std: 0.02571, params: {'reg_alpha': 100}],
 {'reg_alpha': 0.01},
 0.7339827091690398)

In [62]:
param_test7 = {
 'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05]
}
gsearch7 = GridSearchCV(estimator = xgboost.XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=3,
 min_child_weight=5, gamma=0.4, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test7, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch7.fit(xtrain_tfidf_ngram_chars.tocsc(), train_y)
gsearch7.grid_scores_, gsearch7.best_params_, gsearch7.best_score_

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.73203, std: 0.02490, params: {'reg_alpha': 0},
  mean: 0.73203, std: 0.02490, params: {'reg_alpha': 0.001},
  mean: 0.73266, std: 0.02433, params: {'reg_alpha': 0.005},
  mean: 0.73398, std: 0.02347, params: {'reg_alpha': 0.01},
  mean: 0.72890, std: 0.02235, params: {'reg_alpha': 0.05}],
 {'reg_alpha': 0.01},
 0.7339827091690398)

In [78]:
xgb3 = xgboost.XGBClassifier(
 learning_rate =0.01,
 n_estimators=1000,
 max_depth=4,
 min_child_weight=7,
 gamma=0.4,
 subsample=0.8,
 colsample_bytree=0.8,
 reg_alpha=0.01,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)

In [79]:
auc = train_model(xgb3, xtrain_tfidf_ngram_chars.tocsc(), train_y, xtest_tfidf_ngram_chars.tocsc())
print ("Xgb, CharLevel Vectors: ", auc)

Xgb, CharLevel Vectors:  0.7004341468668065


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
